In [100]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import json


In [2]:
pediaters = pd.read_csv("data/pediater_doctors.csv")
pediaters

,title,doctor_url,specialization,kraj,okres,pediater,kraj_id,okres_id
0,"Všeobecná ambulancia pre deti a dorast, Bratis...",https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,Pediatria,Bratislavsky Kraj,Bratislava 1,1,1,3
1,"Ambulancia detskej psychiatrie, Bratislava-Sta...",https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,Detská psychiatria,Bratislavsky Kraj,Bratislava 1,1,1,3
2,"Ambulancia rýchlej lekárskej pomoci, Bratislav...",https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,Rýchla lekárska pomoc,Bratislavsky Kraj,Bratislava 1,1,1,3
3,"Ambulancia rýchlej zdravotnej pomoci, Bratisla...",https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,Rýchla zdravotná pomoc,Bratislavsky Kraj,Bratislava 1,1,1,3
4,"Ambulancia Hasičského záchranného zboru, Radli...",https://www.e-vuc.sk/bsk/zdravotnictvo/ambulan...,Urgentná medicína,Bratislavsky Kraj,Bratislava 1,1,1,3
...,...,...,...,...,...,...,...,...
2147,"Všeobecná ambulancia pre deti a dorast, Humenn...",https://www.e-vuc.sk/psk/zdravotnictvo/ambulan...,Všeobecná starostlivosť o deti a dorast,Prešovský,Humenné,1,4,17
2148,"Všeobecná ambulancia pre deti a dorast, Humenn...",https://www.e-vuc.sk/psk/zdravotnictvo/ambulan...,Všeobecná starostlivosť o deti a dorast,Prešovský,Humenné,1,4,17
2149,"Všeobecná ambulancia pre deti a dorast, Dagmar...",https://www.e-vuc.sk/psk/zdravotnictvo/ambulan...,Všeobecná starostlivosť o deti a dorast,Prešovský,Humenné,1,4,17
2150,Ambulancia pediatrickej pneumológie a ftizeoló...,https://www.e-vuc.sk/psk/zdravotnictvo/ambulan...,Pediatrická pneumológia a ftizeológia,Prešovský,Humenné,1,4,17


In [126]:
df_rows = []
for index,pediater in pediaters.iterrows():

    print(index,"/","2152")
    url = pediater['doctor_url']
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    table = soup.find('table', attrs={'class': 'data_table'})
    regex = "[^\w ]"
    rows = table.find_all('tr')

    info_dict = {}
    doctor_id = ""

    for row in rows:
        if row.find('th') != None:
            th = row.find('th').text.strip()
            th = re.sub(regex, "", th, count=0, flags=0).strip()

            td = row.find('td').text
            td_rows = []
            for td_row in td.split("\n"):
                r = td_row.strip()
                if r != "":
                    td_rows.append(r)

            info_dict[th] = "\n".join(td_rows)
            if th == "Identifikátor":
                doctor_id = td_rows[0]

    tables_hours = soup.find_all(
        'table', attrs={"class": 'data_table no-full-width'})
    table_hours_text = []
    for hour_table in tables_hours:
        for tr in hour_table.find_all('tr'):
            if tr.find('th') != None:
                th = tr.find('th').text.strip()
            if tr.find('td') != None:
                td = tr.find('td').text.strip()
            table_hours_text.append(re.sub(r"\s ", "", th) + " " + re.sub(r"\s ", "", td))

    table_hours_text = "\n".join(table_hours_text)

    map = soup.find('a', attrs={'title': 'Mapa'})

    lat, lng = map['href'].split(":")[2].split(",")[0], map['href'].split(":")[2].split(",")[1]
    df_row = {
        "doctor_id" : doctor_id,
        "info" :  info_dict,
        "open_hours" : table_hours_text,
        "lat" : lat,
        "lng" : lng
    }
    df_rows.append(df_row)

0 / 2152
1 / 2152
2 / 2152
3 / 2152
4 / 2152
5 / 2152
6 / 2152
7 / 2152
8 / 2152
9 / 2152
10 / 2152
11 / 2152
12 / 2152
13 / 2152
14 / 2152
15 / 2152
16 / 2152
17 / 2152
18 / 2152
19 / 2152
20 / 2152
21 / 2152
22 / 2152
23 / 2152
24 / 2152
25 / 2152
26 / 2152
27 / 2152
28 / 2152
29 / 2152
30 / 2152
31 / 2152
32 / 2152
33 / 2152
34 / 2152
35 / 2152
36 / 2152
37 / 2152
38 / 2152
39 / 2152
40 / 2152
41 / 2152
42 / 2152
43 / 2152
44 / 2152
45 / 2152
46 / 2152
47 / 2152
48 / 2152
49 / 2152
50 / 2152
51 / 2152
52 / 2152
53 / 2152
54 / 2152
55 / 2152
56 / 2152
57 / 2152
58 / 2152
59 / 2152
60 / 2152
61 / 2152
62 / 2152
63 / 2152
64 / 2152
65 / 2152
66 / 2152
67 / 2152
68 / 2152
69 / 2152
70 / 2152
71 / 2152
72 / 2152
73 / 2152
74 / 2152
75 / 2152
76 / 2152
77 / 2152
78 / 2152
79 / 2152
80 / 2152
81 / 2152
82 / 2152
83 / 2152
84 / 2152
85 / 2152
86 / 2152
87 / 2152
88 / 2152
89 / 2152
90 / 2152
91 / 2152
92 / 2152
93 / 2152
94 / 2152
95 / 2152
96 / 2152
97 / 2152
98 / 2152
99 / 2152
100 / 2152

In [169]:
df2 = pd.DataFrame(df_rows)

In [170]:
df2['adresa'] = df2['info'].apply(lambda row :  row['Miesto prevádzkovania'])

In [171]:
df2

,doctor_id,info,open_hours,lat,lng,adresa
0,61-54051363-A0001,{'Druh zariadenia': 'všeobecná ambulancia pre ...,Poskytovateľ platnosť od: 01.10.2022\nPondelok...,48.1546935,17.1159519,"prízemie, samostatný trakt, m.č. 2.2a\nPolikli..."
1,61-53545940-A0002,{'Druh zariadenia': 'ambulancia detskej psychi...,Poskytovateľ platnosť od: 01.06.2021\nPondelok...,48.1459930,17.1025367,m.č. 6\nZochova\n754/6 - 8\n81103\nBratislava ...
2,72-17336210-A0041,{'Druh zariadenia': 'ambulancia rýchlej lekárs...,,48.1485965,17.1077477,1\nBratislava - mestská časť Staré Mesto
3,72-17336210-A0197,{'Druh zariadenia': 'ambulancia rýchlej zdravo...,,48.1485965,17.1077477,2\nBratislava - mestská časť Staré Mesto
4,51-00151866-A0053,{'Druh zariadenia': 'špecializovaná ambulancia...,,48.1501518,17.1148218,Radlinského\n2791/6\n81107\nBratislava - mests...
...,...,...,...,...,...,...
2147,67-44878389-A0001,{'Druh zariadenia': 'všeobecná ambulancia pre ...,,48.9336003,21.9060838,Štefánikova\n1525/29\n06601\nHumenné
2148,67-44624298-A0001,{'Druh zariadenia': 'všeobecná ambulancia pre ...,Email platnosť od: 01.11.2017\nPondelok: 7:00-...,48.9336003,21.9060838,Štefánikova\n1525/29\n06601\nHumenné
2149,67-36499820-A0003,{'Druh zariadenia': 'všeobecná ambulancia pre ...,Telefón platnosť od: 19.10.2020\nPondelok: 7:0...,48.9336266,21.9061044,1.poschodie\nŠtefánikova\n1525\n06601\nHumenné
2150,67-36501824-A0001,{'Druh zariadenia': 'ambulancia pediatrickej p...,,48.9335953,21.8991525,Nemocničná\n2430/41\n06601\nHumenné


In [172]:
zilina_pediaters = df2.join(pediaters)
zilina_pediaters[zilina_pediaters['kraj_id'] == 7].to_csv("zilina_data.csv", index=False)